##Constraints to be considered:


1.Brokerage Fees: Factor in the brokerage fees charged by trading platforms for each trade.

2.Capital Limitations: In a real-world scenario, you cannot buy sequentially without selling first due to limited capital.

3.Market Momentum: Consider the momentum behind a trade. For instance, if a stock price is Rs 35 and rising towards Rs 40, you will likely need to place a buy order at Rs 36 or Rs 37 when your algorithm triggers a buy. Similarly, for selling, the price might be lower than anticipated.

4.Tax Implications: Account for both short-term and long-term capital gains tax.

#Dataset chosen - SOLANA
Solana (SOL) is a one of the cryptocurrencies with high market capitalisations. I have chosen this product as the graph show many serious trends , so we might be able to get many productive entry and exit points.This is a historic data for the last 2 years, this was downloaded from investing.com .

#Strategy
I have already explored with the basic trading metrics like the RSI,EMA,Bollinger bands,etc. to get the basic idea of the trends and the strategy to use.

From the analaysis I have chosen to adapt the following strategy:

Let say we have a capital of 2000, I am going to split it between to plans

1.Short-term - 30%

2.Long-term - 70%

##Short-Term Strategy
I propose the combine the MACD and RSI , using the MACD strategy we can find the movement of recent prices with respect to longer history and also the general trend.

if( MACD>0 and Signal>0 -> general uptrend)

if( MACD<0 and Signal<0 -> general downtrend)
and we can generate buy/sell signals with the MACD strategy as well,
with RSI we can find if the stock is overbought or underbought, so combining these strategy we can find whether to enter or exit.

For this strategy i have chosen to trade in only the general uptrend since this less risky and more profitable

## Long Term Strategy
I chose the safest option for this , we take the EMA50 and EMA200 ,
we can generate buy/sell signals with position of EMA50 with respect to the EMA 200

EMA50 > EMA200 ->signals surge in recent prices and a good entry point
EMA50 < EMA200 ->Signals drastic drop in prices and a exit point

with RSI we can find if the product is overbought or underbought so combining these techniques we will be able maximize profit with minimum risk.

EMA50 X EMA200 is a relatively safer option for long term trades and combining it RSI makes it even safer.

##General buying or selling strategy
When in a buying zone , i choose to invest only 50% of the total capital percent as this gives us the cusion so that even if we lose this trade we might be able to get back with rest of the money we have.
Or from another point we could even get a better entry point than before.

When in selling zone, with the short term strategy choose to sell only if my position_loss is more than 10% or if i have achieved a position_profit is above 25%, that too i only sell 50% percent of the asset present. With the long term strategy i choose to sell only if my loss is more than 10% or if profit is more than 60%,that too we sell only 50% of the asset

#Import libraries

In [50]:
import pandas as pd
import numpy as np

#Load DATA

In [51]:
df = pd.read_csv('Solana Historical Data (1).csv')

In [52]:
df.drop(['Vol.','Change %'],axis=1,inplace=True)

In [53]:
df['Date'] = pd.to_datetime(df['Date'],format="%d-%m-%Y")
df.sort_values(by='Date',inplace=True)

In [54]:
df.rename(columns={'Price':'Close'},inplace=True)

In [55]:
total_capital = 2000

##MACD X RSI (Short-term)

In [56]:
def calculate_RSI(df, window=14):
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [57]:
df_MACD_RSI = df.copy()

In [58]:
df_MACD_RSI['MACD'] = df_MACD_RSI['Close'].ewm(span=12, adjust=False).mean() - df_MACD_RSI['Close'].ewm(span=26, adjust=False).mean()
df_MACD_RSI['Signal'] = df_MACD_RSI['MACD'].ewm(span=9, adjust=False).mean()

In [59]:
df_MACD_RSI['RSI'] = calculate_RSI(df_MACD_RSI)

Buy when MACD>Signal and sell when MACD<Signal

In [60]:
df_MACD_RSI['Buy/Sell_MACD'] = np.where(df_MACD_RSI['MACD'] > df_MACD_RSI['Signal'],'Buy','Sell')

In [61]:
#general uptrend
df_MACD_RSI_positive = df_MACD_RSI[(df_MACD_RSI['MACD']>0)&(df_MACD_RSI['Signal']>0)]
#uptrend buy
df_MACD_RSI_positive_Buy = df_MACD_RSI_positive[df_MACD_RSI_positive['Buy/Sell_MACD'] == 'Buy']
#uptrend Sell
df_MACD_RSI_positive_Sell = df_MACD_RSI_positive[df_MACD_RSI_positive['Buy/Sell_MACD'] == 'Sell']

In [62]:
df_MACD_RSI_positive_Buy['RSI'].describe()

count    181.000000
mean      72.848848
std        9.987632
min       44.335519
25%       66.758051
50%       72.436369
75%       79.268717
max       96.632733
Name: RSI, dtype: float64

In [63]:
df_MACD_RSI_positive_Sell['RSI'].describe()

count    166.000000
mean      49.012740
std        9.983235
min       21.988660
25%       42.057556
50%       48.557125
75%       55.968592
max       72.975811
Name: RSI, dtype: float64

We see that the average RSI during the uptrend buy is high due to a over bought market and the average RSI during the uptrend Sell is much lower due to a underbought market
We set the buy and sell signals adjusting to the market

I have chosen the RSI threshold for buy to be 60 since the market is the uptrend buy state and so the average RSI is high so we increase the threshold

I chosen the RSI threshold for sell to 60 as well because examining the dataset where we have sell signal with MACD we have relatively lower RSI throughout so as to adjust with that i have reduced the RSI

Also setting these threshold gives us enough entries and exits

In [64]:
#Dataset holding the buy and sell zones
df_EXt_positive = pd.concat([df_MACD_RSI_positive_Buy[df_MACD_RSI_positive_Buy['RSI']<60],df_MACD_RSI_positive_Sell[df_MACD_RSI_positive_Sell['RSI']>60]])

In [65]:
df_EXt_positive.sort_values(by='Date',ascending=True,inplace=True)

Simulating the strategy with the constraint mentioned

In [66]:
# Initial capital and trading variables
brokerage_fee = 0.003
l_tax = 0.12 #Long term tax
s_tax = 0.20 #Short term tax
capital = total_capital * 0.3 # 30 percent of capital of short term
sol_bought = 0 #Available_SOL
total_invested = 0 #Total invested
fraction_to_invest = 0.5  # 50% of capital to use for each buy order
fraction_to_sell = 0.5    # 50% of stock to sell in each sell order

Total_tax_paid = 0

#Dataframe to hold the Entry_Exit data
df_EntryExit = pd.DataFrame(columns=['Date', 'Entry/Exit', 'Close', 'Traded_SOL', 'Capital', 'Average_Cost', 'Available_SOL', 'Position_Profit/Loss', 'Trade_Type'])

# List to store dictionaries that hold the buy order details
buys = []

# Function to check if a trade is long or short
def determine_trade_type(entry_date, exit_date):
    holding_period = (exit_date - entry_date).days
    return 'Long' if holding_period > 365 else 'Short'


for i in range(len(df_EXt_positive)):
    if df_EXt_positive['Buy/Sell_MACD'].iloc[i] == 'Buy':
        if capital > 50:
            Date = df_EXt_positive['Date'].iloc[i]
            Entry_Exit = "Entry"
            Price = df_EXt_positive['Close'].iloc[i]
            # Execute a buy trade with a fraction of capital
            capital_to_invest = capital * fraction_to_invest
            sol_bought_this_trade = capital_to_invest / Price
            sol_bought += sol_bought_this_trade
            total_invested += capital_to_invest

            capital -= capital_to_invest - (capital_to_invest*brokerage_fee)

            # Store the buy order details
            buys.append({'date': Date, 'quantity': sol_bought_this_trade, 'price': Price})

            # Calculate average cost
            avg_cost = total_invested / sol_bought

            profit_percent = (((sol_bought * Price) - (sol_bought * avg_cost)) / (sol_bought * avg_cost)) * 100

            df_EntryExit = pd.concat([df_EntryExit, pd.DataFrame({'Date': [Date], 'Entry/Exit': [Entry_Exit], 'Close': [Price], 'Traded_SOL': [sol_bought_this_trade], 'Capital': [capital],'Average_Cost': [avg_cost],'Available_SOL': [sol_bought],'Position_Profit/Loss': [profit_percent],'Trade_Type': ['N/A'] })], ignore_index=True)

    elif df_EXt_positive['Buy/Sell_MACD'].iloc[i] == 'Sell':
        if sol_bought > 0:
            # Calculate profit percentage
            Price = df_EXt_positive['Close'].iloc[i]
            profit_percent = (((sol_bought * Price) - (sol_bought * avg_cost)) / (sol_bought * avg_cost)) * 100

            # Execute a sell trade with a fraction of the assets based on profit percentage thresholds
            if profit_percent < -10 or profit_percent > 25:
                Date = df_EXt_positive['Date'].iloc[i]
                Entry_Exit = "Exit"
                sol_to_sell = sol_bought * fraction_to_sell
                sol_bought -= sol_to_sell

                sol_sold = 0
                #List for the various type of trades that could have happened during the sell
                trade_types = []
                total_buy_cost = 0 # for calculation of avg buy cost while selling to calculate profit so as to cut tax
                while sol_sold < sol_to_sell and buys:
                    #get the first buy order
                    buy = buys.pop(0)
                    if sol_sold + buy['quantity'] <= sol_to_sell:
                        trade_type = determine_trade_type(buy['date'], Date)
                        total_buy_cost += buy['quantity'] * buy['price']
                        #append the trade type to the list
                        trade_types.append(trade_type)
                        sol_sold += buy['quantity']
                    else:
                        remaining_qty = sol_to_sell - sol_sold
                        trade_type = determine_trade_type(buy['date'], Date)
                        trade_types.append(trade_type)
                        #calculate the remaining quantity
                        buy['quantity'] -= remaining_qty
                        total_buy_cost += remaining_qty * buy['price']
                        #push it once again to the front of the list
                        buys.insert(0, buy)
                        sol_sold += remaining_qty


                if trade_types.count('Long') > trade_types.count('Short'):
                    trade_type = 'Long'
                else:
                    trade_type = 'Short'

                Avg_buy_price = total_buy_cost / sol_to_sell
                Profit_per_sell = (sol_to_sell * Price) - (Avg_buy_price * sol_to_sell)
                #Tax deductions
                if Profit_per_sell > 0:
                  if trade_type == 'Long':
                      capital += sol_to_sell * Price - (Profit_per_sell * l_tax)
                      Total_tax_paid += (Profit_per_sell * l_tax)
                  else:
                      capital += sol_to_sell * Price - (Profit_per_sell*s_tax)
                      Total_tax_paid += (Profit_per_sell * s_tax)
                else:
                  capital += sol_to_sell * Price

                # Recalculate the total invested
                total_invested -= sol_to_sell * avg_cost

                # Log the trade in df_EntryExit
                df_EntryExit = pd.concat([df_EntryExit, pd.DataFrame({'Date': [Date], 'Entry/Exit': [Entry_Exit], 'Close': [Price], 'Traded_SOL': [sol_to_sell], 'Capital': [capital],'Average_Cost': [avg_cost],'Available_SOL': [sol_bought],'Position_Profit/Loss': [profit_percent],'Trade_Type': [trade_type]})], ignore_index=True)


<ipython-input-66-6400943c1664>:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_EntryExit = pd.concat([df_EntryExit, pd.DataFrame({'Date': [Date], 'Entry/Exit': [Entry_Exit], 'Close': [Price], 'Traded_SOL': [sol_bought_this_trade], 'Capital': [capital],'Average_Cost': [avg_cost],'Available_SOL': [sol_bought],'Position_Profit/Loss': [profit_percent],'Trade_Type': ['N/A'] })], ignore_index=True)


In [67]:
df_EntryExit

,Date,Entry/Exit,Close,Traded_SOL,Capital,Average_Cost,Available_SOL,Position_Profit/Loss,Trade_Type
0,2022-11-06,Entry,32.627,9.194839,300.900000,32.627000,9.194839,0.000000,N/A
1,2022-11-07,Entry,29.599,5.082942,150.901350,31.549021,14.277781,-6.180924,N/A
2,2023-01-29,Exit,26.097,7.138890,337.204970,31.549021,7.138890,-17.281110,Short
3,2023-02-01,Exit,24.986,3.569445,426.391127,31.549021,3.569445,-20.802614,Short
4,2023-02-19,Entry,24.869,8.572744,213.835150,26.832729,12.142189,-7.318410,N/A
5,2023-02-21,Entry,24.980,4.280127,107.238328,26.349855,16.422316,-5.198718,N/A
6,2023-02-22,Entry,24.162,2.219153,53.780021,26.089404,18.641469,-7.387689,N/A
7,2023-02-24,Exit,23.080,9.320734,268.902568,26.089404,9.320734,-11.534966,Short
8,2023-04-19,Entry,22.683,5.927403,134.854638,24.765234,15.248137,-8.407890,N/A
9,2023-04-20,Entry,22.183,3.039594,67.629601,24.336042,18.287732,-8.847133,N/A


For the above simulation I have calculated the tax only for profits incurred,Long_Term_tax = 12% and Short_Term_Tax=20% and i have set the the brokerage_fee to be 0.003% of the total_buying_cost.

while selling the stock i chose the strategy of eliminating the asset that the bought the earliest, i calculate the duration between the sell date and the earliest available bought date for tax calculation

In [68]:
df_EntryExit.to_csv('ShortTermEntryExit.csv',index=False)

In [69]:
Total_tax_paid

304.1119895838676

Total tax paid is $304

### Returns
Capital = available_capital + (Available_SOL*Todays_price)

Capital = 1704.226 + (0.16x140)

Capital = 1726.62

#EMA X RSI (Long term)

In [70]:
df_EMA_RSI = df.copy()

In [71]:
df_EMA_RSI['EMA50'] = df_EMA_RSI['Close'].ewm(span=50, adjust=False).mean()
df_EMA_RSI['EMA200'] = df_EMA_RSI['Close'].ewm(span=200, adjust=False).mean()
df_EMA_RSI['RSI'] = calculate_RSI(df_EMA_RSI)

In [72]:
df_EMA_RSI['Buy/Sell_EMA'] = np.where(df_EMA_RSI['EMA50'] > df_EMA_RSI['EMA200'],'Buy','Sell')
df_EMA_RSI['Buy/Sell_RSI'] = np.where(df_EMA_RSI['RSI'] < 30,'Buy',(np.where(df_EMA_RSI['RSI'] > 70,'Sell','Hold')))

Since this strategy is for long term we adapt the most commonly used RSI threshold

In [73]:
df_BuySell = df_EMA_RSI[((df_EMA_RSI['Buy/Sell_EMA'] == 'Buy') & (df_EMA_RSI['Buy/Sell_RSI'] == 'Buy'))|((df_EMA_RSI['Buy/Sell_EMA'] == 'Sell') & (df_EMA_RSI['Buy/Sell_RSI'] == 'Sell'))]

In [74]:
df_BuySell.sort_values(by='Date',ascending=True,inplace=True)

<ipython-input-74-eb5f224c02de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BuySell.sort_values(by='Date',ascending=True,inplace=True)


In [75]:
# Initial capital and trading variables
brokerage_fee = 0.003
l_tax = 0.12 #Long term tax
s_tax = 0.20 #Short term tax
capital = total_capital * 0.7 # 30 percent of capital of short term
sol_bought = 0 #Available_SOL
total_invested = 0 #Total invested
fraction_to_invest = 0.5  # 50% of capital to use for each buy order
fraction_to_sell = 0.5    # 50% of stock to sell in each sell order

Total_tax_paid = 0

#Dataframe to hold the Entry_Exit data
df_EntryExit = pd.DataFrame(columns=['Date', 'Entry/Exit', 'Close', 'Traded_SOL', 'Capital', 'Average_Cost', 'Available_SOL', 'Position_Profit/Loss', 'Trade_Type'])

# List to store dictionaries that hold the buy order details
buys = []

# Function to check if a trade is long or short
def determine_trade_type(entry_date, exit_date):
    holding_period = (exit_date - entry_date).days
    return 'Long' if holding_period > 365 else 'Short'


for i in range(len(df_BuySell)):
    if df_BuySell['Buy/Sell_EMA'].iloc[i] == 'Buy':
        if capital > 50:
            Date = df_BuySell['Date'].iloc[i]
            Entry_Exit = "Entry"
            Price = df_BuySell['Close'].iloc[i]
            # Execute a buy trade with a fraction of capital
            capital_to_invest = capital * fraction_to_invest
            sol_bought_this_trade = capital_to_invest / Price
            sol_bought += sol_bought_this_trade
            total_invested += capital_to_invest

            capital -= capital_to_invest - (capital_to_invest*brokerage_fee)

            # Store the buy order details
            buys.append({'date': Date, 'quantity': sol_bought_this_trade, 'price': Price})

            # Calculate average cost
            avg_cost = total_invested / sol_bought

            profit_percent = (((sol_bought * Price) - (sol_bought * avg_cost)) / (sol_bought * avg_cost)) * 100

            df_EntryExit = pd.concat([df_EntryExit, pd.DataFrame({'Date': [Date], 'Entry/Exit': [Entry_Exit], 'Close': [Price], 'Traded_SOL': [sol_bought_this_trade], 'Capital': [capital],'Average_Cost': [avg_cost],'Available_SOL': [sol_bought],'Position_Profit/Loss': [profit_percent],'Trade_Type': ['N/A'] })], ignore_index=True)

    elif df_BuySell['Buy/Sell_EMA'].iloc[i] == 'Sell':
        if sol_bought > 0:
            # Calculate profit percentage
            Price = df_BuySell['Close'].iloc[i]
            profit_percent = (((sol_bought * Price) - (sol_bought * avg_cost)) / (sol_bought * avg_cost)) * 100

            # Execute a sell trade with a fraction of the assets based on profit percentage thresholds
            if profit_percent < -10 or profit_percent > 25:
                Date = df_BuySell['Date'].iloc[i]
                Entry_Exit = "Exit"
                sol_to_sell = sol_bought * fraction_to_sell
                sol_bought -= sol_to_sell

                sol_sold = 0
                #List for the various type of trades that could have happened during the sell
                trade_types = []
                total_buy_cost = 0 # for calculation of avg buy cost while selling to calculate profit so as to cut tax
                while sol_sold < sol_to_sell and buys:
                    #get the first buy order
                    buy = buys.pop(0)
                    if sol_sold + buy['quantity'] <= sol_to_sell:
                        trade_type = determine_trade_type(buy['date'], Date)
                        total_buy_cost += buy['quantity'] * buy['price']
                        #append the trade type to the list
                        trade_types.append(trade_type)
                        sol_sold += buy['quantity']
                    else:
                        remaining_qty = sol_to_sell - sol_sold
                        trade_type = determine_trade_type(buy['date'], Date)
                        trade_types.append(trade_type)
                        #calculate the remaining quantity
                        buy['quantity'] -= remaining_qty
                        total_buy_cost += remaining_qty * buy['price']
                        #push it once again to the front of the list
                        buys.insert(0, buy)
                        sol_sold += remaining_qty


                if trade_types.count('Long') > trade_types.count('Short'):
                    trade_type = 'Long'
                else:
                    trade_type = 'Short'

                Avg_buy_price = total_buy_cost / sol_to_sell
                Profit_per_sell = (sol_to_sell * Price) - (Avg_buy_price * sol_to_sell)
                #Tax deductions
                if Profit_per_sell > 0:
                  if trade_type == 'Long':
                      capital += sol_to_sell * Price - (Profit_per_sell * l_tax)
                      Total_tax_paid += (Profit_per_sell * l_tax)
                  else:
                      capital += sol_to_sell * Price - (Profit_per_sell*s_tax)
                      Total_tax_paid += (Profit_per_sell * s_tax)
                else:
                  capital += sol_to_sell * Price

                # Recalculate the total invested
                total_invested -= sol_to_sell * avg_cost

                # Log the trade in df_EntryExit
                df_EntryExit = pd.concat([df_EntryExit, pd.DataFrame({'Date': [Date], 'Entry/Exit': [Entry_Exit], 'Close': [Price], 'Traded_SOL': [sol_to_sell], 'Capital': [capital],'Average_Cost': [avg_cost],'Available_SOL': [sol_bought],'Position_Profit/Loss': [profit_percent],'Trade_Type': [trade_type]})], ignore_index=True)


<ipython-input-75-4a5a9badf9e4>:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_EntryExit = pd.concat([df_EntryExit, pd.DataFrame({'Date': [Date], 'Entry/Exit': [Entry_Exit], 'Close': [Price], 'Traded_SOL': [sol_bought_this_trade], 'Capital': [capital],'Average_Cost': [avg_cost],'Available_SOL': [sol_bought],'Position_Profit/Loss': [profit_percent],'Trade_Type': ['N/A'] })], ignore_index=True)


In [76]:
df_EntryExit

,Date,Entry/Exit,Close,Traded_SOL,Capital,Average_Cost,Available_SOL,Position_Profit/Loss,Trade_Type
0,2023-08-22,Entry,20.584,34.006996,702.100000,20.584000,34.006996,0.000000,N/A
1,2023-08-25,Entry,20.420,17.191479,352.103150,20.528932,51.198475,-0.530626,N/A
2,2023-08-26,Entry,20.288,8.677621,176.579730,20.494015,59.876096,-1.005242,N/A
3,2023-08-28,Entry,20.568,4.292584,88.554734,20.498964,64.168680,0.336779,N/A
4,2024-04-12,Entry,153.281,0.288864,44.410199,21.094021,64.457544,626.656130,N/A


We got very good entry points with the strategy, the capital if the assets were sold would be =(Available_SOLxPriceToday)+available_capital

Approx sol price today - $140

So capital - 140x64.45 + 44.4

capital = 9067.4

# Total gain from the strategy

Short term capital = 1726.62
Long term capital = 9067.4

Gain = 9067.4 + 1726.62 - 2000

Gain = 8794.02

Gain% = 8794.02/2000 x 100

Gain% = 439.7%

In [77]:
df_EntryExit.to_csv('LongtermEntryExit.csv',index=False)